In [1]:
from Angrybird import AngryBird
import numpy as np
import matplotlib.pyplot as plt
#import tensorflow as tf
from tqdm.notebook import tqdm

In [2]:
env = AngryBird()
env.reset()
for i in range(11):
    print(env.step(np.zeros(2)))

(array([ 1.        , 10.62066026, 17.9348062 ]), 0.0, False, None)
(array([ 2.        , 22.07286858, 30.79423652]), 0.0, False, None)
(array([ 3.        , 34.1063792 , 38.36176627]), 0.0, False, None)
(array([ 4.        , 47.03640483, 40.67747271]), 0.0, False, None)
(array([ 5.        , 61.30443218, 38.0988979 ]), 0.0, False, None)
(array([ 6.        , 76.07770513, 30.17268727]), 0.0, False, None)
(array([ 7.        , 90.83431402, 18.1529957 ]), 0.0, False, None)
(array([  8.        , 107.08724705,   0.47372754]), 0.0, False, None)
(array([  9.        , 125.5124703 , -22.87753652]), 0.0, False, None)
(array([ 10.        , 145.63854672, -51.56595679]), 0.0, False, None)
(array([ 11.        , 145.63854672, -51.56595679]), 18914.99810289746, True, None)


In [3]:
env.j_prime_MC()

0.0